In [ ]:
import sys
sys.path.append('..')  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import fuzzy_systems as fs
from fuzzy_systems.learning import WangMendelClassification

In [4]:
# URL do dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt'

# Carregar dados
columns = ['area', 'perimeter', 'compactness', 'length', 'width', 'asymmetry', 'groove_length', 'variety']
df = pd.read_csv(url, sep='\s+', names=columns)

# Converter classes para 0, 1, 2
df['variety'] = df['variety'] - 1

print(f"Shape: {df.shape}")
print(f"\nClasses: {df['variety'].value_counts().sort_index()}")
df.head()

Shape: (210, 8)

Classes: variety
0    70
1    70
2    70
Name: count, dtype: int64


,area,perimeter,compactness,length,width,asymmetry,groove_length,variety
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0


In [5]:
# Usar as 4 features mais discriminativas
selected_features = ['area', 'perimeter', 'compactness', 'groove_length']

X = df[selected_features].values
y = df['variety'].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (210, 4)
y shape: (210,)


In [6]:
n_classes = 3
y_onehot = np.zeros((len(y), n_classes))
for i, label in enumerate(y):
    y_onehot[i, label] = 1

print(f"y_onehot shape: {y_onehot.shape}")
print(f"\nExemplo:")
print(f"y[0] = {y[0]} -> y_onehot[0] = {y_onehot[0]}")

y_onehot shape: (210, 3)

Exemplo:
y[0] = 0 -> y_onehot[0] = [1. 0. 0.]


In [7]:
X_train, X_test, y_train_onehot, y_test_onehot, y_train, y_test = train_test_split(
    X, y_onehot, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

Train: (147, 4), Test: (63, 4)


In [10]:
system = fs.MamdaniSystem(name="Iris One-vs-All")
input_names = ['comp_sepala', 'larg_sepala', 'comp_petala', 'larg_petala']

universes = [
        (np.min(X[:,0]), np.max(X[:,0])),  # comprimento sépala
        (np.min(X[:,1]), np.max(X[:,1])),  # largura sépala
        (np.min(X[:,2]), np.max(X[:,2])),  # comprimento pétala
        (np.min(X[:,3]), np.max(X[:,3])),  # largura pétala
    ]

for name, universe in zip(input_names, universes):
    system.add_input(name, universe)
    # 3 partições por feature
    system.add_term(name, 'pequeno', 'triangular',
                    (universe[0], universe[0], (universe[0]+universe[1])/2))
    system.add_term(name, 'medio', 'triangular',
                    (universe[0], (universe[0]+universe[1])/2, universe[1]))
    system.add_term(name, 'grande', 'triangular',
                    ((universe[0]+universe[1])/2, universe[1], universe[1]))

# Adicionar 3 saídas binárias
output_names = ['is_Kama', 'is_Rosa', 'is_Canadian']
for name in output_names:
    system.add_output(name, (0, 1))
    system.add_term(name, 'nao', 'triangular', (0,0,1.0))
    system.add_term(name, 'sim', 'triangular', (0, 1, 1.0))

print(f"   ✅ Sistema criado:")
print(f"      Entradas: {list(system.input_variables.keys())}")
print(f"      Saídas: {list(system.output_variables.keys())}")
print()


   ✅ Sistema criado:
      Entradas: ['comp_sepala', 'larg_sepala', 'comp_petala', 'larg_petala']
      Saídas: ['is_Kama', 'is_Rosa', 'is_Canadian']



In [14]:
wm = WangMendelClassification(
        system=system,
        X_train=X_train,
        y_train=y_train_onehot,
    )

system_trained = wm.fit(verbose=True)

WANG-MENDEL CLASSIFICAÇÃO - GERANDO REGRAS

📊 Configuração:
   • Entradas:  4 (['comp_sepala', 'larg_sepala', 'comp_petala', 'larg_petala'])
   • Saídas:    3 (['is_Kama', 'is_Rosa', 'is_Canadian'])
   • Amostras:  147

   🔍 Detectado: Classificação One-Hot Encoding
      → Regras serão geradas considerando todas as saídas juntas

REGRAS GERADAS POR SAÍDA

📤 Saída: is_Kama
   ✅ Regras geradas: 18
   ⚠️  Conflitos resolvidos: 25

   📋 Exemplos de regras (mostrando 5):
      1. IF comp_sepala = pequeno AND larg_sepala = medio AND comp_petala = medio AND larg_petala = medio
         THEN is_Kama = nao (grau=0.207)
      2. IF comp_sepala = pequeno AND larg_sepala = pequeno AND comp_petala = medio AND larg_petala = pequeno
         THEN is_Kama = nao (grau=0.492)
      3. IF comp_sepala = pequeno AND larg_sepala = pequeno AND comp_petala = pequeno AND larg_petala = medio
         THEN is_Kama = nao (grau=0.478)
      4. IF comp_sepala = pequeno AND larg_sepala = pequeno AND comp_petala = m

In [18]:
wm.predict(X_test)

array([1, 2, 2, 2, 1, 2, 0, 1, 2, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0,
       2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 1, 2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2,
       0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 2, 1, 1, 1])

In [21]:
for i in range(len(X_test)):
    # Criar entrada
    inputs = {
        'comp_sepala': X_test[i, 0],
        'larg_sepala': X_test[i, 1],
        'comp_petala': X_test[i, 2],
        'larg_petala': X_test[i, 3]
    }

    # Obter saídas
    output = system_trained.evaluate(inputs)

In [23]:
confusion_matrix(y_test, wm.predict(X_test))

array([[16,  0,  5],
       [10, 11,  0],
       [ 2,  0, 19]])

#### Seção 2

In [43]:
fis2 = fs.MamdaniSystem(name="Iris")
input_names = ['comp_sepala', 'larg_sepala', 'comp_petala', 'larg_petala']

universes = [
        (np.min(X[:,0]), np.max(X[:,0])),  # comprimento sépala
        (np.min(X[:,1]), np.max(X[:,1])),  # largura sépala
        (np.min(X[:,2]), np.max(X[:,2])),  # comprimento pétala
        (np.min(X[:,3]), np.max(X[:,3])),  # largura pétala
    ]

for name, universe in zip(input_names, universes):
    fis2.add_input(name, universe)
    # 3 partições por feature
    fis2.add_term(name, 'pequeno', 'triangular',
                    (universe[0], universe[0], (universe[0]+universe[1])/2))
    fis2.add_term(name, 'medio', 'triangular',
                    (universe[0], (universe[0]+universe[1])/2, universe[1]))
    fis2.add_term(name, 'grande', 'triangular',
                    ((universe[0]+universe[1])/2, universe[1], universe[1]))

# Adicionar 3 saídas binárias
output_names = ['is_Kama', 'is_Rosa', 'is_Canadian']
fis2.add_output('Tipo', (-1, 3))
fis2.add_term('Tipo', 'is_Kama', 'triangular', (0,1,2))
fis2.add_term('Tipo', 'is_Rosa', 'triangular',(1,2,3))
fis2.add_term('Tipo', 'is_Canadian', 'triangular',(-1,0,1) )


print(f"   ✅ Sistema criado:")
print(f"      Entradas: {list(fis2.input_variables.keys())}")
print(f"      Saídas: {list(fis2.output_variables.keys())}")
print()


   ✅ Sistema criado:
      Entradas: ['comp_sepala', 'larg_sepala', 'comp_petala', 'larg_petala']
      Saídas: ['Tipo']



In [44]:
wm2= WangMendelClassification(
        system=fis2,
        X_train=X_train,
        y_train=y_train,
    )

wm2.fit(verbose=True)

WANG-MENDEL CLASSIFICAÇÃO - GERANDO REGRAS

📊 Configuração:
   • Entradas:  4 (['comp_sepala', 'larg_sepala', 'comp_petala', 'larg_petala'])
   • Saídas:    1 (['Tipo'])
   • Amostras:  147

REGRAS GERADAS POR SAÍDA

📤 Saída: Tipo
   ✅ Regras geradas: 18
   ⚠️  Conflitos resolvidos: 129

   📋 Exemplos de regras (mostrando 5):
      1. IF comp_sepala = pequeno AND larg_sepala = medio AND comp_petala = medio AND larg_petala = medio
         THEN Tipo = is_Canadian (grau=0.207)
      2. IF comp_sepala = pequeno AND larg_sepala = pequeno AND comp_petala = medio AND larg_petala = pequeno
         THEN Tipo = is_Canadian (grau=0.492)
      3. IF comp_sepala = pequeno AND larg_sepala = pequeno AND comp_petala = pequeno AND larg_petala = medio
         THEN Tipo = is_Canadian (grau=0.478)
      4. IF comp_sepala = pequeno AND larg_sepala = pequeno AND comp_petala = medio AND larg_petala = medio
         THEN Tipo = is_Canadian (grau=0.341)
      5. IF comp_sepala = medio AND larg_sepala = medi

MamdaniSystem(name='Iris', inputs=4, outputs=1, rules=18)

In [45]:
confusion_matrix(y_test, wm2.predict(X_test).round())

array([[ 5, 16,  0],
       [ 0, 10, 11],
       [19,  2,  0]])

In [35]:
wm2.predict_membership(X_test)

[[{'is_Kama': 1.1102230246251565e-16,
   'is_Rosa': 0.9999999999999999,
   'is_Canadian': 0.0}],
 [{'is_Kama': 0.0,
   'is_Rosa': 0.28030615462247543,
   'is_Canadian': 0.7196938453775246}],
 [{'is_Kama': 0.0,
   'is_Rosa': 0.51786411250174,
   'is_Canadian': 0.48213588749826}],
 [{'is_Kama': 0.0,
   'is_Rosa': 0.7453098372973073,
   'is_Canadian': 0.25469016270269274}],
 [{'is_Kama': 0.2832049826331341,
   'is_Rosa': 0.7167950173668659,
   'is_Canadian': 0.0}],
 [{'is_Kama': 0.0,
   'is_Rosa': 0.47995418198033435,
   'is_Canadian': 0.5200458180196657}],
 [{'is_Kama': 0.733344408932288,
   'is_Rosa': 0.26665559106771203,
   'is_Canadian': 0.0}],
 [{'is_Kama': 0.4923686279222613,
   'is_Rosa': 0.5076313720777387,
   'is_Canadian': 0.0}],
 [{'is_Kama': 0.0,
   'is_Rosa': 0.9207299166423697,
   'is_Canadian': 0.0792700833576303}],
 [{'is_Kama': 0.020779873089058154,
   'is_Rosa': 0.9792201269109418,
   'is_Canadian': 0.0}],
 [{'is_Kama': 0.49392904376396307,
   'is_Rosa': 0.50607095623603